In [20]:
import json
import os
import re
from typing import List, Dict

import pandas as pd

In [21]:
# 사용할 엔티티 타입
ENTITY_TYPES = ["PS", "LC", "OG", "DT", "TI", "QT"]

# <|PS|>엔티티<|PS|> 같은 패턴 찾는 정규식
ENTITY_PATTERN = re.compile(
    r"<\|(PS|LC|OG|DT|TI|QT)\|>(.+?)<\|\1\|>",
    flags=re.DOTALL
)

In [22]:
def parse_tagged_sentence(tagged_text: str) -> Dict:
    """
    라벨링된 문장 하나를 받아서
    - 태그가 제거된 실제 문장 text
    - 엔티티 스팬 리스트 [{"start": int, "end": int, "label": "PS"}, ...]
    를 반환합니다.
    """
    entities = []
    clean_text_parts = []
    curr_pos = 0  # clean_text 상의 현재 길이

    last_end = 0
    for match in ENTITY_PATTERN.finditer(tagged_text):
        label = match.group(1)
        surface = match.group(2)

        # 태그 이전의 일반 텍스트 추가
        before = tagged_text[last_end:match.start()]
        if before:
            clean_text_parts.append(before)
            curr_pos += len(before)

        # 엔티티 텍스트 추가
        ent_start = curr_pos
        clean_text_parts.append(surface)
        curr_pos += len(surface)
        ent_end = curr_pos

        entities.append({
            "start": ent_start,
            "end": ent_end,
            "label": label
        })

        last_end = match.end()

    # 마지막 태그 뒤에 남은 텍스트
    tail = tagged_text[last_end:]
    if tail:
        clean_text_parts.append(tail)
        curr_pos += len(tail)

    clean_text = "".join(clean_text_parts)

    return {
        "text": clean_text,
        "entities": entities
    }

In [23]:
# ---- 예시 테스트 ----
sample = """“반갑다. 힘찬 호랑이 해야” <|DT|>2022년<|DT|> 임인년(壬寅年) 첫날이 밝았다."""
parsed = parse_tagged_sentence(sample)
print(parsed["text"])
print(parsed["entities"])

for ent in parsed['entities']:
    print(parsed['text'][ent['start']:ent['end']])

“반갑다. 힘찬 호랑이 해야” 2022년 임인년(壬寅年) 첫날이 밝았다.
[{'start': 17, 'end': 22, 'label': 'DT'}]
2022년


# 1. Process data

In [54]:
dataset_name = "NIKL_NEWSPAPER_2023_CSV"

fname = "NEWSPAPER_2022_1"
sample_name = "sample1"
sample_name = "sample2"

fname = "NEWSPAPER_2022_2"
sample_name = "sample3"

# fname = "NEWSPAPER_2022_3"
# sample_name = "sample4"

save_base_path = f"results/{dataset_name}-{fname}-{sample_name}"

In [55]:
df = pd.read_parquet(f"data/{dataset_name}-{fname}-{sample_name}.parquet")

In [56]:
df.head()

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_ids,sentence_offsets,text
79120,NLRW2300000005,NLRW2300000005.1969,대구신문 2022년 기사,이상호,대구신문,20220213,사회,"경제>취업_창업, 사회>교육_시험, 사회>여성","[""NLRW2300000005.1969.1"", ""NLRW2300000005.1969...","[[0, 17], [18, 62], [63, 120], [121, 172], [17...",포스코 취업아카데미 교육생 모집 포스코 취업아카데미 교육생 모집이 지난 10일 시작...
93389,NLRW2300000006,NLRW2300000006.14810,부산일보 2022년 기사,김길수,부산일보,20220626,사회,"경제>취업_창업, 지역>경남, 지역>대구","[""NLRW2300000006.14810.1"", ""NLRW2300000006.148...","[[0, 36], [37, 123], [124, 264], [265, 325], [...","경남도, 28일 CECO에서 자동차·기계·항공산업 채용박람회 개최 경남도는 고용위기..."
45425,NLRW2300000003,NLRW2300000003.1606,경북일보 2022년 기사,김형소 기자,경북일보,20220214,사회,"지역>충남, 지역>울산, 지역>대전","[""NLRW2300000003.1606.1"", ""NLRW2300000003.1606...","[[0, 36], [37, 100], [101, 209], [210, 302], [...","산불 예방·진화 공동체계 구축…울진군, 비행훈련원 등과 협약 체결 울진군은 지난 1..."
2356,NIRW2300000002,NIRW2300000002.46542,뉴스핌 2022년 기사,성소의,뉴스핌,20221026,사회,None,"[""NIRW2300000002.46542.1"", ""NIRW2300000002.465...","[[0, 31], [32, 81], [82, 129], [130, 285], [28...",8월 출생아수 2만1758명...77개월째 ‘사상 최저’ 지난 8월 출생아 수가 2...
94138,NLRW2300000006,NLRW2300000006.15485,부산일보 2022년 기사,이성훈,부산일보,20220704,사회,사회>사회일반,"[""NLRW2300000006.15485.1"", ""NLRW2300000006.154...","[[0, 24], [25, 82], [83, 189], [190, 289], [29...","경남대, 제1회 국토대장정 성공적으로 마무리 경남대학교는 ‘지역사랑 나라사랑’을 주..."


In [57]:
import re

LABELS = ["PS", "LC", "OG", "DT", "TI", "QT"]

# Allowed tags only
TAG_PATTERN = re.compile(r"<\|(PS|LC|OG|DT|TI|QT)\|>")
# Any tag-like pattern, used to catch unknown labels
ANY_TAG_PATTERN = re.compile(r"<\|([^|]+)\|>")

def validate(text: str):
    """
    Validate custom tags in `text`.

    Returns:
        is_valid (bool): True if all checks pass.
        counts (dict): {label: count} for each allowed label.
        errors (list[str]): Human-readable error messages.
    """
    counts = {label: 0 for label in LABELS}
    errors = []
    stack = []  # will hold labels in nesting order, e.g. ["PS", "LC"]

    # 1) Check for unknown label tags like <|XX|> where XX not in LABELS
    for m in ANY_TAG_PATTERN.finditer(text):
        label = m.group(1)
        if label not in LABELS:
            errors.append(f"Unknown label '{label}' at position {m.start()}.")

    # 2) Process allowed tags in order, maintain counts and nesting stack
    for m in TAG_PATTERN.finditer(text):
        label = m.group(1)
        counts[label] += 1

        # Nesting / pairing logic:
        # - If top of stack is same label, this tag closes that span -> pop.
        # - Otherwise, this tag opens a new span -> push.
        if stack and stack[-1] == label:
            stack.pop()  # close span
        else:
            stack.append(label)  # open span

    # 3) After processing, stack should be empty (all spans closed)
    if stack:
        # Remaining items in stack are unclosed spans
        # Show them from outermost to innermost for clarity
        unclosed = " -> ".join(stack)
        errors.append(f"Unmatched / unclosed tags in nesting order: {unclosed}")

    # 4) Ensure each label appears an even number of times (paired)
    for label, count in counts.items():
        if count % 2 != 0:
            errors.append(
                f"Label '{label}' appears {count} times (must be even, in pairs)."
            )

    is_valid = len(errors) == 0
    return is_valid

In [58]:
def process_df(df):
    remove_ids = []
    entities = []
    texts = []

    invalid_count = 0
    for i in range(df.shape[0]):
        row = df.iloc[i]
        
        docid = row['doc_id']
        
        if not os.path.exists(os.path.join(save_base_path, f"response/{docid}.json")):
            remove_ids.append(docid)
            continue
        
        with open(os.path.join(save_base_path, f"generated/{docid}.json"), "r") as f:
            generated = json.load(f)
        
        text = generated['tagged_text']
        
        if not validate(text):
            remove_ids.append(docid)
            
            invalid_count+=1
            continue
        
        
        parsed = parse_tagged_sentence(text)
        row_entities = parsed['entities']
        
        texts.append(parsed['text'])
        entities.append(row_entities)

    print(f"Removed Ids {len(remove_ids)} Invalid {invalid_count}")
    df = df[~df.doc_id.isin(remove_ids)]
    df['text']=texts
    df['entities'] = entities
    df = df.drop(labels=['sentence_offsets'], axis=1)
    return df

In [59]:
print(f"Original {df.shape[0]}")
df = process_df(df)
print(f"Processed {df.shape[0]}")

Original 14996
Removed Ids 1161 Invalid 1161
Processed 13835


/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_26731/1910398991.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text']=texts
/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_26731/1910398991.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['entities'] = entities


In [60]:
df.head()

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_ids,text,entities
93389,NLRW2300000006,NLRW2300000006.14810,부산일보 2022년 기사,김길수,부산일보,20220626,사회,"경제>취업_창업, 지역>경남, 지역>대구","[""NLRW2300000006.14810.1"", ""NLRW2300000006.148...","경남도, 28일 CECO에서 자동차·기계·항공산업 채용박람회 개최 경남도는 고용위기...","[{'start': 0, 'end': 3, 'label': 'OG'}, {'star..."
45425,NLRW2300000003,NLRW2300000003.1606,경북일보 2022년 기사,김형소 기자,경북일보,20220214,사회,"지역>충남, 지역>울산, 지역>대전","[""NLRW2300000003.1606.1"", ""NLRW2300000003.1606...",울진군은 지난 10일 지역 관계 기관들과 합동으로 산불 예방과 진화 등 공동체계 구...,"[{'start': 0, 'end': 3, 'label': 'LC'}, {'star..."
2356,NIRW2300000002,NIRW2300000002.46542,뉴스핌 2022년 기사,성소의,뉴스핌,20221026,사회,None,"[""NIRW2300000002.46542.1"", ""NIRW2300000002.465...",8월 출생아수 2만1758명...77개월째 ‘사상 최저’ 지난 8월 출생아 수가 2...,"[{'start': 0, 'end': 2, 'label': 'DT'}, {'star..."
94138,NLRW2300000006,NLRW2300000006.15485,부산일보 2022년 기사,이성훈,부산일보,20220704,사회,사회>사회일반,"[""NLRW2300000006.15485.1"", ""NLRW2300000006.154...","경남대, 제1회 국토대장정 성공적으로 마무리 경남대학교는 ‘지역사랑 나라사랑’을 주...","[{'start': 0, 'end': 3, 'label': 'OG'}, {'star..."
58801,NLRW2300000004,NLRW2300000004.13645,남도일보 2022년 기사,김명식 기자,남도일보,20220906,사회,"지역>충남, 지역>경기, 사회>교육_시험","[""NLRW2300000004.13645.1"", ""NLRW2300000004.136...",‘전면 대면수업’에 학폭도 증가...광주시교육청 실태조사 결과 광주지역 학교폭력이 ...,"[{'start': 20, 'end': 26, 'label': 'OG'}, {'st..."


# 2. Save to disk as datasets

In [61]:
from datasets import Dataset

In [62]:
# raw_dataset = Dataset.from_list([parsed, parsed])
ds = Dataset.from_pandas(df)

In [63]:
ds = ds.remove_columns(["__index_level_0__"])

In [64]:
ds

Dataset({
    features: ['file_id', 'doc_id', 'title', 'author', 'publisher', 'date', 'topic', 'original_topic', 'sentence_ids', 'text', 'entities'],
    num_rows: 13835
})

In [65]:
ds[0]

{'file_id': 'NLRW2300000006',
 'doc_id': 'NLRW2300000006.14810',
 'title': '부산일보 2022년 기사',
 'author': '김길수',
 'publisher': '부산일보',
 'date': 20220626,
 'topic': '사회',
 'original_topic': '경제>취업_창업, 지역>경남, 지역>대구',
 'sentence_ids': '["NLRW2300000006.14810.1", "NLRW2300000006.14810.2", "NLRW2300000006.14810.3", "NLRW2300000006.14810.4", "NLRW2300000006.14810.5", "NLRW2300000006.14810.6", "NLRW2300000006.14810.7"]',
 'text': '경남도, 28일 CECO에서 자동차·기계·항공산업 채용박람회 개최 경남도는 고용위기산업 퇴직자 재취업을 위한 ‘2022년 자동차·기계·항공산업 채용박람회’를 오는 28일 창원컨벤션센터(CECO)에서 연다고 26일 밝혔다. 경남고용안정선제대응지원센터와 함께 개최하는 이번 행사에는 자동차, 기계, 항공 등 고용위기산업 관련 기업인 경한코리아 등 17개 사가 참여한다. 이들 기업은 박람회에서 연구개발직, 생산관리직, 기술설계직, 생산기술직 등 100여 명을 채용할 예정이다. 박람회에는 현장에서 구인기업과 구직자 간 면접까지 진행해 실질적인 채용의 장이 될 것으로 경남도는 기대했다. 구직자를 지원하기 위해 증명사진 무료 촬영, 스트레스 검사, 직업타로로 알아보는 적성검사, 취업서류·이미지·면접 컨설팅 등 다양한 부대행사도 함께 진행한다. 2020년 6월 개소한 경남고용안정선제대응지원센터는 그동안 지역 제조업 퇴직자와 구직자 3034명을 대상으로 취업지원서비스를 제공했다. 이 중 1506명이 재취업하는 성과를 거뒀다. 경남도와 경남고용안정선제대응지원센터는 하반기에 김해에서도 채용박람회를 열 예정이다.',
 'entities

In [66]:
sample = ds[0]

text = sample['text']
for ent in sample['entities']:
    print(ent['label'], text[ent['start']:ent['end']])

OG 경남도
DT 28일
LC CECO
OG 자동차·기계·항공산업
OG 경남도
DT 2022년
OG 자동차·기계·항공산업
DT 28일
LC 창원컨벤션센터(CECO)
DT 26일
OG 경남고용안정선제대응지원센터
OG 경한코리아
QT 17개 사
QT 100여 명
OG 경남도
OG 증명사진
OG 스트레스 검사
OG 직업타로
DT 2020년 6월
OG 경남고용안정선제대응지원센터
QT 3034명
QT 1506명
OG 경남도
OG 경남고용안정선제대응지원센터
DT 하반기
LC 김해


In [67]:
dataset_dir = f"../dataset/{dataset_name}-{fname}-{sample_name}.parquet"
ds.to_parquet(dataset_dir)

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

45237099

In [68]:
# ds.save_to_disk(dataset_dir)